# 🚀 Kaggle Remote VS Code Setup

Connect VS Code to Kaggle using zrok's private tunnel (150GB/month free).

## Quick Start
1. Get your zrok token from [https://zrok.io](https://zrok.io)
2. Run the cell below with your token
3. On your local machine: `python local/connect.py --token YOUR_ZROK_TOKEN`

---

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("zrok")

In [ ]:
# Clone (or update) and run setup
import os
if os.path.exists('/kaggle/working/kaggle-vscode-zrok'):
    %cd /kaggle/working/kaggle-vscode-zrok
    !git fetch --quiet && git reset --hard origin/main --quiet
else:
    !git clone --quiet https://github.com/mehtab78/kaggle-vscode-zrok.git
    %cd /kaggle/working/kaggle-vscode-zrok

# Run with your token (edit the token below)
!python3 setup_script.py --token token --password "0"
# Optional: Add SSH key authentication
# !python3 setup_script.py --token "<YOUR_ZROK_TOKEN>" --password "0" --authorized_keys_url "<public_key_url>"

---

## 📖 Local Machine Setup

### Prerequisites
- Install [zrok](https://docs.zrok.io/docs/guides/install/)
- Install [VS Code](https://code.visualstudio.com/) with [Remote-SSH](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-ssh) extension

### Connect
```bash
# Clone this repo
git clone https://github.com/mehtab78/kaggle-vscode-zrok.git
cd kaggle-vscode-zrok/local

# Connect (auto-discovers tunnel and opens VS Code)
python connect.py --token YOUR_ZROK_TOKEN

# Or skip VS Code launch
python connect.py --token YOUR_ZROK_TOKEN --no-vscode
```

---

## 🛠️ Utility Cells

In [ ]:
# Check status
!echo "=== SSH ===" && ps aux | grep -v grep | grep sshd && echo "\n=== ZROK ===" && zrok status

In [ ]:
# System info
print("=" * 40)
!echo "CPU:  $(cat /proc/cpuinfo | grep 'model name' | head -1 | cut -d: -f2)"
!echo "RAM:  $(free -h | awk '/Mem:/ {print $2}')"
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader 2>/dev/null || echo "GPU:  None"
!echo "Disk: $(df -h /kaggle/working | awk 'NR==2 {print $4}') free"
print("=" * 40)